In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/city_data.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,haibara-akanedai,34.5333,135.9500,31.81,52,3,3.08,JP,1691718838
1,saint-philippe,-21.3585,55.7679,19.77,78,1,2.15,RE,1691718685
2,waitangi,-43.9535,-176.5597,12.12,89,100,3.13,NZ,1691718644
3,whitehorse,60.7161,-135.0538,20.05,76,75,1.03,CA,1691718631
4,ilulissat,69.2167,-51.1000,7.01,81,75,1.54,GL,1691718651


In [3]:
%%capture --no-display
# Configure the map plot
latitude = city_data_df["Lat"].iloc[0]
longitude = city_data_df["Lng"].iloc[0]
m = folium.Map(location=[latitude, longitude], zoom_start=2)

# Add markers for each city with the size of the point based on humidity
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row["Lat"], row["Lng"]],
        radius=row["Humidity"] / 5, 
        color="blue",
        fill=True,
        fill_color="blue",
        popup=f"City: {row['City']}<br>Humidity: {row['Humidity']}%",
    ).add_to(m)

# Display the map
m.save("humidity_map.html")  
display(m)

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 30) &
    (city_data_df["Humidity"] < 50)
].copy()  # Make a copy of the filtered DataFrame to avoid the warning

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data
sample_data = filtered_cities_df.sample(10)  # Display a sample of 10 rows
sample_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
270,deschutes river woods,43.9915,-121.3584,27.45,21,0,6.69,US,1691718904
515,shahrud,36.4182,54.9763,21.62,31,0,2.69,IR,1691718966
204,charters towers,-20.1000,146.2667,24.26,34,4,5.15,AU,1691718889
185,port hedland,-20.3167,118.5667,29.03,21,1,3.09,AU,1691718884
323,gazojak,41.1875,61.4036,24.48,38,0,3.87,TM,1691718919
421,birjand,32.8663,59.2211,21.26,24,0,0.00,IR,1691718942
40,saynshand,44.8824,110.1163,24.82,49,33,3.41,MN,1691718849
504,zhangye,38.9342,100.4517,26.30,40,0,2.30,CN,1691718963
343,williams lake,52.1415,-122.1445,23.92,38,75,3.09,CA,1691718653
288,espanola,35.9911,-106.0806,29.98,31,100,5.25,US,1691718909


In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df 
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
sample_data = hotel_df.sample(10)  # Display a sample of 10 rows
sample_data

,City,Country,Lat,Lng,Humidity,Hotel Name
315,kabul,AF,34.5000,69.4167,31,
206,illizi,DZ,26.4833,8.4667,16,
412,arhribs,DZ,36.7936,4.3116,34,
446,maracai,BR,-22.6106,-50.6672,41,
243,cuencame de ceniceros,MX,24.8711,-103.6964,42,
229,danghara,TJ,38.0958,69.3400,27,
194,petropavlovsk-kamchatsky,RU,53.0452,158.6483,49,
71,vilhena,BR,-12.7406,-60.1458,37,
148,pirenopolis,BR,-15.8525,-48.9592,44,
149,ain oussera,DZ,35.4514,2.9058,29,


In [6]:
# Set a larger radius for hotel search
radius = 10000

# Set other parameters for the Geoapify API request
params = {
    "filter": "category=hotels",
    "limit": 1,
    "apiKey": "geoapify_key"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"category=hotels&near={lat},{lng}&radius={radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
sample_data = hotel_df.sample(10)  # Display a sample of 10 rows
sample_data


Starting hotel search
mwene-ditu - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
saynshand - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
vilhena - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
afif - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
south hedland - nearest hotel: No hotel found
pirenopolis - nearest hotel: No hotel found
ain oussera - nearest hotel: No hotel found
port hedland - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
charters towers - nearest hotel: No hotel found
illizi - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
danghara - nearest hotel: No hotel found
darab - nearest hotel: No hotel found
taco pozo - nearest hotel: No hotel found
cuencame de ceniceros - nearest hotel: No hotel found
sabha - nearest hotel: No hotel found
deschutes river woods - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
229,danghara,TJ,38.0958,69.3400,27,No hotel found
40,saynshand,MN,44.8824,110.1163,49,No hotel found
106,karratha,AU,-20.7377,116.8463,35,No hotel found
71,vilhena,BR,-12.7406,-60.1458,37,No hotel found
446,maracai,BR,-22.6106,-50.6672,41,No hotel found
204,charters towers,AU,-20.1000,146.2667,34,No hotel found
206,illizi,DZ,26.4833,8.4667,16,No hotel found
321,diamantino,BR,-14.4086,-56.4461,36,No hotel found
70,tamanrasset,DZ,22.7850,5.5228,12,No hotel found
227,ust-nera,RU,64.5667,143.2000,35,No hotel found


In [ ]:
# Create sample data for demonstration
cities = ['City A', 'City B', 'City C']
humidity = [75, 60, 40]

# Configure the map plot
plt.figure(figsize=(10, 6))
plt.scatter(range(len(cities)), humidity, color='blue')
plt.title('City Humidity')
plt.xlabel('Cities')
plt.ylabel('Humidity')
plt.xticks(range(len(cities)), cities, rotation=45)

# Display the map
plt.tight_layout()

# Save the plot to a file (optional)
plt.savefig('humidity_map.png')

# Close the plot to prevent direct output
plt.close()